In [161]:
import os, sys, json, time, math, itertools, random, textwrap
from datetime import datetime, timedelta, date
from pathlib import Path

import numpy as np
import pandas as pd

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (9, 5)
plt.rcParams["axes.grid"] = True

# seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Load Datasets

In [224]:
# load the official dataset
visitation = pd.read_csv("data/VisitationData.csv")
climate = pd.read_csv("data/ClimdateData.csv")

In [163]:
# check the dataset info()
visitation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            165 non-null    float64
 1   Week            165 non-null    float64
 2   Mt. Baw Baw     165 non-null    float64
 3   Mt. Stirling    165 non-null    float64
 4   Mt. Hotham      165 non-null    float64
 5   Falls Creek     165 non-null    float64
 6   Mt. Buller      165 non-null    float64
 7   Selwyn          165 non-null    float64
 8   Thredbo         165 non-null    float64
 9   Perisher        165 non-null    float64
 10  Charlotte Pass  165 non-null    float64
dtypes: float64(11)
memory usage: 86.0 KB


In [164]:
climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39813 entries, 0 to 39812
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Bureau of Meteorology station number  39813 non-null  int64  
 1   Year                                  39813 non-null  int64  
 2   Month                                 39813 non-null  int64  
 3   Day                                   39813 non-null  int64  
 4   Maximum temperature (Degree C)        38275 non-null  float64
 5   Minimum temperature (Degree C)        38280 non-null  float64
 6   Rainfall amount (millimetres)         37857 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 2.1 MB


In [165]:
# load external datasets like pricing, accessibility, special features
life_features = pd.read_csv("data/ticket_lift_fees.csv")
vic_snowfall = pd.read_csv("data/vic_snowfall.csv")
nsw_snowfall = pd.read_csv("data/nsw_snowfall.csv")
distance = pd.read_csv("data/ski_resorts_distances.csv")

In [166]:
life_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ski Resort     9 non-null      object 
 1   parking        9 non-null      int64  
 2   lift           9 non-null      int64  
 3   blue_slope     9 non-null      float64
 4   red_slope      9 non-null      float64
 5   black_slope    9 non-null      float64
 6   cross-country  9 non-null      bool   
dtypes: bool(1), float64(3), int64(2), object(1)
memory usage: 573.0+ bytes


In [167]:
vic_snowfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Station                      424 non-null    int64  
 1   Date                         424 non-null    object 
 2   Day                          424 non-null    object 
 3   Average_snow_deepth_10years  424 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 13.4+ KB


In [168]:
nsw_snowfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Station                      33 non-null     int64  
 1   Season                       33 non-null     object 
 2   Total Snowfall (inches)      33 non-null     int64  
 3   Snowfall Days                33 non-null     int64  
 4   Average Base Depth (inches)  28 non-null     float64
 5   Max Base Depth (inches)      28 non-null     float64
 6   Biggest Snowfall (inches)    33 non-null     int64  
 7   Total Snowfall (cm)          33 non-null     float64
 8   Average Base Depth (cm)      28 non-null     float64
 9   Max Base Depth (cm)          28 non-null     float64
 10  Biggest Snowfall (cm)        33 non-null     float64
dtypes: float64(6), int64(4), object(1)
memory usage: 3.0+ KB


In [169]:
distance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ski Resort         9 non-null      object 
 1   State              9 non-null      object 
 2   Distance_from_MEL  9 non-null      object 
 3   Distance_from_SYD  9 non-null      object 
 4   Distance_from_CAN  9 non-null      object 
 5   Time_from_MEL      9 non-null      float64
 6   Time_from_SYD      9 non-null      float64
 7   Time_from_CAN      9 non-null      float64
 8   Public Transport   9 non-null      object 
dtypes: float64(3), object(6)
memory usage: 780.0+ bytes


# Data Cleaning

In [170]:
# ---- CONFIG / CONSTANTS ----
SEED = 42

RESORT_TO_STATION = {
    "Thredbo":         71032,  # Thredbo AWS
    "Perisher":        71075,  # Perisher AWS
    "Selwyn":          72161,  # Cabramurra SMHEA AWS (closest for Selwyn)
    "Charlotte Pass":  71075,  # often use Perisher AWS; change if you prefer 72161
    "Mt. Buller":      83024,  # Mount Buller
    "Falls Creek":     83084,  # Falls Creek
    "Mt. Hotham":      83085,  # Mount Hotham
    "Mt. Baw Baw":     85291,  # Mount Baw Baw
    "Mt. Stirling":    83024,  # share with Buller (no dedicated BoM AWS)
}

# Standard resort name canonicalization (match across all frames)
RESORT_CANON = {
    "Mt. Baw Baw": "Mt. Baw Baw",
    "Mt. Stirling": "Mt. Stirling",
    "Mt. Hotham": "Mt. Hotham",
    "Falls Creek": "Falls Creek",
    "Mt. Buller": "Mt. Buller",
    "Selwyn": "Selwyn",
    "Thredbo": "Thredbo",
    "Perisher": "Perisher",
    "Charlotte Pass": "Charlotte Pass",
}

In [171]:
# your start / end (month, day only)
start_month, start_day = 6, 9
end_month, end_day     = 9, 22

# build a datetime for easy comparisons (any year)
climate["date"] = pd.to_datetime(climate[["Year","Month","Day"]])

def in_range(row, start_m, start_d, end_m, end_d):
    # build dates for that specific year
    year = row["Year"]
    start = pd.Timestamp(year=year, month=start_m, day=start_d)
    end   = pd.Timestamp(year=year, month=end_m,   day=end_d)
    return start <= row["date"] <= end

climate_clean = climate[climate.apply(in_range, axis=1,
                                      args=(start_month, start_day, end_month, end_day))]
climate_clean = climate_clean[climate_clean["Year"]>=2014]
climate_clean.to_csv("intermediate/climate_snowseason.csv")

### Week keys and Distance Parsing

In [172]:
# Week key from pandas Timestamp
def _week_key_from_date(dt):
    iso = dt.isocalendar()
    return int(iso.year), int(iso.week)

# Parse strings like "383 km / 4h50m" -> (383.0, 290.0)
def _parse_km_time(cell):
    if pd.isna(cell): 
        return np.nan, np.nan
    s = str(cell)
    try:
        km_part, time_part = [x.strip() for x in s.split('/')]
        km = float(km_part.lower().replace('km','').strip())
        t = time_part.lower().replace(' ', '')
        h = 0; m = 0
        if 'h' in t:
            h_str = t.split('h')[0]
            h = int(''.join(filter(str.isdigit, h_str))) if h_str else 0
            t = t.split('h',1)[1]
        if 'm' in t:
            m_str = t.split('m')[0]
            m = int(''.join(filter(str.isdigit, m_str))) if m_str else 0
        minutes = h*60 + m
        km = float(km)
        return km, float(minutes)
    except Exception:
        return np.nan, np.nan


In [173]:
vis_df = visitation.copy()

# keep valid year/week
vis_df = vis_df.dropna(subset=["Year","Week"]).copy()
vis_df["Year"] = vis_df["Year"].astype(int)
vis_df["Week"] = vis_df["Week"].astype(int)

resort_cols = [c for c in vis_df.columns if c not in ["Year","Week"]]
vis_long = vis_df.melt(id_vars=["Year","Week"], value_vars=resort_cols,
                       var_name="Ski Resort", value_name="visitors")
vis_long["Ski Resort"] = vis_long["Ski Resort"].map(RESORT_CANON).fillna(vis_long["Ski Resort"])
vis_long = vis_long.dropna(subset=["visitors"]).reset_index(drop=True)

vis_long.head()

,Year,Week,Ski Resort,visitors
0,2014,1,Mt. Baw Baw,555.0
1,2014,2,Mt. Baw Baw,804.0
2,2014,3,Mt. Baw Baw,993.0
3,2014,4,Mt. Baw Baw,2976.0
4,2014,5,Mt. Baw Baw,11112.0


In [174]:
# --- VIC snowfall weekly aggregation (10-year averages) ---

sf = vic_snowfall.copy()

# 0) Parse '9-Jun' style dates by appending a dummy year
BASE_YEAR = 2014
sf["date"] = pd.to_datetime(
    sf["Date"].astype(str).str.strip() + f"-{BASE_YEAR}",
    format="%d-%b-%Y",   # use "%d-%B-%Y" if months are full names like 'June'
    errors="coerce"
)

# 1) Season weeks relative to Jun-09 (same BASE_YEAR)
season_start = pd.Timestamp(BASE_YEAR, 6, 9)
season_end   = pd.Timestamp(BASE_YEAR, 9, 30)  # wide enough to include your last rows

sf = sf.loc[(sf["date"] >= season_start) & (sf["date"] <= season_end)].copy()
sf["Week"] = ((sf["date"] - season_start).dt.days // 7 + 1).clip(1, 15).astype(int)

# 2) Standardize columns / dtypes
sf = sf.rename(columns={"Station": "station_id"})
sf["station_id"] = pd.to_numeric(sf["station_id"], errors="coerce").astype("Int64")
sf["Average_snow_deepth_10years"] = pd.to_numeric(sf["Average_snow_deepth_10years"], errors="coerce")

# 3) Weekly snow depth (10-year avg) per station (no Year dimension)
snowfall_weekly_by_station = (
    sf.groupby(["Week", "station_id"], as_index=False)["Average_snow_deepth_10years"].mean()
)

# 4) Merge into climate table for every Year (repeat the weekly pattern each year)
climate = climate_weekly_by_station.copy()
climate["station_id"] = pd.to_numeric(climate["station_id"], errors="coerce").astype("Int64")
climate["Year"] = climate["Year"].astype(int)
climate["Week"] = climate["Week"].astype(int)

climate = climate.merge(
    snowfall_weekly_by_station,
    on=["Week", "station_id"],
    how="left"
)

# 5) Optional: smooth with a 3-week rolling avg within each Year & station
climate = (climate
    .sort_values(["station_id", "Year", "Week"])
    .assign(
        SnowAvgDepth_cm_roll3=lambda d:
            d.groupby(["station_id", "Year"])["Average_snow_deepth_10years"]
             .transform(lambda s: s.rolling(3, min_periods=1).mean())
    )
)

display(climate)


,Year,Week,station_id,TempMax_C,TempMin_C,Rain_mm,Average_snow_deepth_10years,SnowAvgDepth_cm_roll3
0,2014,1,71032,4.714286,-2.000000,40.6,NaN,NaN
7,2014,2,71032,2.657143,-1.685714,8.0,NaN,NaN
14,2014,3,71032,-1.028571,-4.114286,4.2,NaN,NaN
21,2014,4,71032,-1.314286,-4.357143,0.4,NaN,NaN
28,2014,5,71032,-1.685714,-5.185714,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1182,2025,4,85291,4.328571,-0.485714,131.6,23.385714,18.395238
1189,2025,5,85291,2.014286,-1.457143,63.2,25.114286,22.695238
1196,2025,6,85291,1.542857,-2.214286,19.8,36.171429,28.223810
1203,2025,7,85291,2.328571,-1.457143,33.4,42.842857,34.709524


In [175]:
# --- Step 1: VIC weekly merge (same as you already do) ---
sf = vic_snowfall.copy()
BASE_YEAR = 2014
sf["date"] = pd.to_datetime(
    sf["Date"].astype(str).str.strip() + f"-{BASE_YEAR}",
    format="%d-%b-%Y", errors="coerce"
)

season_start = pd.Timestamp(BASE_YEAR, 6, 9)
season_end   = pd.Timestamp(BASE_YEAR, 9, 30)

sf = sf.loc[(sf["date"] >= season_start) & (sf["date"] <= season_end)].copy()
sf["Week"] = ((sf["date"] - season_start).dt.days // 7 + 1).clip(1, 15).astype(int)

sf = sf.rename(columns={"Station": "station_id"})
sf["station_id"] = pd.to_numeric(sf["station_id"], errors="coerce").astype("Int64")
sf["Average_snow_deepth_10years"] = pd.to_numeric(sf["Average_snow_deepth_10years"], errors="coerce")

snowfall_weekly_by_station = (
    sf.groupby(["Week","station_id"], as_index=False)["Average_snow_deepth_10years"].mean()
)

climate = climate_weekly_by_station.copy()
climate["station_id"] = pd.to_numeric(climate["station_id"], errors="coerce").astype("Int64")
climate["Year"] = climate["Year"].astype(int)
climate["Week"] = climate["Week"].astype(int)

climate = climate.merge(snowfall_weekly_by_station, on=["Week","station_id"], how="left")

# --- Step 2: NSW season-level merge ---
nsw = nsw_snowfall.copy()
nsw = nsw.rename(columns={"Average Base Depth (cm)": "AvgBaseDepth_cm",
                          "Station": "station_id"})
nsw["station_id"] = pd.to_numeric(nsw["station_id"], errors="coerce").astype("Int64")

import re
def season_to_year(s):
    m = re.search(r"(\d{4})", str(s))
    return int(m.group(1)) if m else None

nsw["Year"] = nsw["Season"].apply(season_to_year).astype("Int64")
nsw_yearly = nsw.groupby(["Year","station_id"], as_index=False)["AvgBaseDepth_cm"].mean()

# merge on Year+station_id (fills NSW resorts, all weeks same value)
climate = climate.merge(nsw_yearly, on=["Year","station_id"], how="left")

# --- Step 3: Combine VIC + NSW into one column ---
if "Average_snow_deepth_10years" not in climate.columns:
    climate["Average_snow_deepth_10years"] = pd.NA

climate["Average_snow_deepth_10years"] = climate["Average_snow_deepth_10years"].fillna(
    climate["AvgBaseDepth_cm"]
)

climate = climate.drop(columns=["AvgBaseDepth_cm"])

# --- Step 4: Rolling average within station/year ---
climate = (
    climate.sort_values(["station_id","Year","Week"])
           .assign(
               SnowAvgDepth_cm_roll3=lambda d:
                   d.groupby(["station_id","Year"])["Average_snow_deepth_10years"]
                    .transform(lambda s: s.rolling(3,min_periods=1).mean())
           )
)

display(climate)


,Year,Week,station_id,TempMax_C,TempMin_C,Rain_mm,Average_snow_deepth_10years,SnowAvgDepth_cm_roll3
0,2014,1,71032,4.714286,-2.000000,40.6,132.080000,132.080000
7,2014,2,71032,2.657143,-1.685714,8.0,132.080000,132.080000
14,2014,3,71032,-1.028571,-4.114286,4.2,132.080000,132.080000
21,2014,4,71032,-1.314286,-4.357143,0.4,132.080000,132.080000
28,2014,5,71032,-1.685714,-5.185714,3.0,132.080000,132.080000
...,...,...,...,...,...,...,...,...
1182,2025,4,85291,4.328571,-0.485714,131.6,23.385714,18.395238
1189,2025,5,85291,2.014286,-1.457143,63.2,25.114286,22.695238
1196,2025,6,85291,1.542857,-2.214286,19.8,36.171429,28.223810
1203,2025,7,85291,2.328571,-1.457143,33.4,42.842857,34.709524


In [176]:
feat_df = pd.read_csv("data/ticket_lift_fees.csv")
df = feat_df
df["Ski Resort"] = df["Ski Resort"].map(RESORT_CANON).fillna(df["Ski Resort"])

num_cols = ["parking", "lift", "blue_slope", "red_slope", "black_slope"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df.dtypes

Ski Resort        object
parking            int64
lift               int64
blue_slope       float64
red_slope        float64
black_slope      float64
cross-country       bool
dtype: object

In [177]:
# Convert cross-country column ---
if "cross-country" in df.columns:
    df["cross_country"] = df["cross-country"].astype(bool).astype(int)
    df = df.drop(columns=["cross-country"])
df.head()


,Ski Resort,parking,lift,blue_slope,red_slope,black_slope,cross_country
0,Falls Creek,67,243,6.0,35.0,8.0,0
1,Mount Buller,67,234,9.5,21.4,16.6,0
2,Mount Hotham,67,243,6.0,12.0,12.0,0
3,Mount Baw Baw,67,89,7.0,3.0,0.0,0
4,Mount Stirling,67,0,0.0,0.0,0.0,1


In [191]:
# --- Cell 1: Start with a copy & canonicalize resort names ---
df = distance.copy()

# assumes RESORT_CANON is a dict like {"Mt. Hotham":"Mount Hotham", ...}
if "Ski Resort" in df.columns:
    df["Ski Resort"] = df["Ski Resort"].map(RESORT_CANON).fillna(df["Ski Resort"])

df.head()


,Ski Resort,State,Distance_from_MEL,Distance_from_SYD,Distance_from_CAN,Time_from_MEL,Time_from_SYD,Time_from_CAN,Public Transport
0,Falls Creek,VIC,383 km,672 km,462 km,4.83,7.87,5.62,SnowBus
1,Mount Buller,VIC,234 km,771 km,561 km,3.25,8.37,6.12,Coach
2,Mount Hotham,VIC,373 km,702 km,493 km,4.62,8.07,5.82,SnowBus
3,Mount Baw Baw,VIC,176 km,914 km,639 km,2.83,10.83,7.73,Train
4,Mount Stirling,VIC,236 km,775 km,566 km,3.72,8.78,6.53,Coach


In [193]:
# --- Cell 2: Helper parser for "383 km / 4h50m" → (383.0, 290) ---
import re

def _km_minutes_from_text(s: str):
    if s is None:
        return (None, None)
    s = str(s)

    # km
    km_match = re.search(r'(\d+(?:\.\d+)?)\s*km', s, flags=re.I)
    km = float(km_match.group(1)) if km_match else None

    # time: supports "4h50m", "4 h 50 m", "4h", "50m"
    h = m = 0
    h_match = re.search(r'(\d+)\s*h', s, flags=re.I)
    m_match = re.search(r'(\d+)\s*m', s, flags=re.I)
    if h_match:
        h = int(h_match.group(1))
    if m_match:
        m = int(m_match.group(1))
    minutes = h * 60 + m if (h or m) else None

    return (km, minutes)


In [194]:
def clean_distances(dist_df):
    """Parse distance/time strings; derive travel metrics; tidy public transport."""
    df = dist_df.copy()
    df["Ski Resort"] = df["Ski Resort"].map(RESORT_CANON).fillna(df["Ski Resort"])

    # Parse distance/time from MEL/SYD/CAN
    for city in ["MEL","SYD","CAN"]:
        km_col = f"Distance_{city}_km"
        vals = df[f"Distance_from_{city}"].apply(parse_km_time)
        df[km_col]  = [v[0] for v in vals]

    # Public transport → boolean/categorical
    if "Public Transport" in df.columns:
        df["public_transport"] = df["Public Transport"].astype(str).str.strip().str.lower()
        df["has_public_transport"] = df["public_transport"].isin(["yes","y","available","true"]).astype(int)

    keep = ["Ski Resort","State",
            "Distance_MEL_km","Distance_SYD_km","Distance_CAN_km"
            ,"has_public_transport"]
    return df[[c for c in keep if c in df.columns]].copy()

In [195]:
# --- Cell 3: Parse the distance/time fields for MEL / SYD / CAN (only if present) ---
for city in ["MEL", "SYD", "CAN"]:
    src_col = f"Distance_from_{city}"
    if src_col in df.columns:
        parsed = df[src_col].apply(_km_minutes_from_text)
        df[f"Distance_{city}_km"]  = [p[0] for p in parsed]

df.head()


,Ski Resort,State,Distance_from_MEL,Distance_from_SYD,Distance_from_CAN,Time_from_MEL,Time_from_SYD,Time_from_CAN,Public Transport,Distance_MEL_km,Distance_SYD_km,Distance_CAN_km
0,Falls Creek,VIC,383 km,672 km,462 km,4.83,7.87,5.62,SnowBus,383.0,672.0,462.0
1,Mount Buller,VIC,234 km,771 km,561 km,3.25,8.37,6.12,Coach,234.0,771.0,561.0
2,Mount Hotham,VIC,373 km,702 km,493 km,4.62,8.07,5.82,SnowBus,373.0,702.0,493.0
3,Mount Baw Baw,VIC,176 km,914 km,639 km,2.83,10.83,7.73,Train,176.0,914.0,639.0
4,Mount Stirling,VIC,236 km,775 km,566 km,3.72,8.78,6.53,Coach,236.0,775.0,566.0


In [196]:
# --- Cell 4: Public transport standardization ---
if "Public Transport" in df.columns:
    pt = df["Public Transport"].astype(str).str.strip().str.lower()
    yes_tokens = {"yes", "y", "available", "true", "1"}
    df["public_transport"] = pt
    df["has_public_transport"] = pt.isin(yes_tokens).astype(int)
elif "has_public_transport" not in df.columns:
    # ensure the column exists even if source column missing
    df["has_public_transport"] = 0

df.head()


,Ski Resort,State,Distance_from_MEL,Distance_from_SYD,Distance_from_CAN,Time_from_MEL,Time_from_SYD,Time_from_CAN,Public Transport,Distance_MEL_km,Distance_SYD_km,Distance_CAN_km,public_transport,has_public_transport
0,Falls Creek,VIC,383 km,672 km,462 km,4.83,7.87,5.62,SnowBus,383.0,672.0,462.0,snowbus,0
1,Mount Buller,VIC,234 km,771 km,561 km,3.25,8.37,6.12,Coach,234.0,771.0,561.0,coach,0
2,Mount Hotham,VIC,373 km,702 km,493 km,4.62,8.07,5.82,SnowBus,373.0,702.0,493.0,snowbus,0
3,Mount Baw Baw,VIC,176 km,914 km,639 km,2.83,10.83,7.73,Train,176.0,914.0,639.0,train,0
4,Mount Stirling,VIC,236 km,775 km,566 km,3.72,8.78,6.53,Coach,236.0,775.0,566.0,coach,0


In [202]:
# --- Cell 5: Keep tidy columns that exist ---
keep = [
    "Ski Resort", "State",
    "Distance_MEL_km", "Distance_SYD_km", "Distance_CAN_km",
    "Time_from_MEL","Time_from_SYD","Time_from_CAN",
    "has_public_transport"
]
cleaned_dist = df[[c for c in keep if c in df.columns]].copy()
cleaned_dist.head()


,Ski Resort,State,Distance_MEL_km,Distance_SYD_km,Distance_CAN_km,Time_from_MEL,Time_from_SYD,Time_from_CAN,has_public_transport
0,Falls Creek,VIC,383.0,672.0,462.0,4.83,7.87,5.62,0
1,Mount Buller,VIC,234.0,771.0,561.0,3.25,8.37,6.12,0
2,Mount Hotham,VIC,373.0,702.0,493.0,4.62,8.07,5.82,0
3,Mount Baw Baw,VIC,176.0,914.0,639.0,2.83,10.83,7.73,0
4,Mount Stirling,VIC,236.0,775.0,566.0,3.72,8.78,6.53,0


# Feature Engineering

In [198]:
clim = climate_df.copy()

# === Basic temperature features ===
clim["TempMean_C"] = (clim["TempMax_C"] + clim["TempMin_C"]) / 2
clim["DiurnalRange_C"] = clim["TempMax_C"] - clim["TempMin_C"]

# Freezing-related indicators
clim["Freeze_flag"] = (clim["TempMin_C"] <= 0).astype(int)
clim["Cold_degree"] = (0 - clim["TempMean_C"]).clip(lower=0)  # how far below 0 °C

# Rainfall transforms
clim["Rain_log1p"] = np.log1p(clim["Rain_mm"].clip(lower=0))
clim["Rain_heavy"] = (clim["Rain_mm"] >= 20).astype(int)

# Interactions
clim["Cold_x_Rain"] = clim["Cold_degree"] * clim["Rain_log1p"]

# Snow depth transforms
clim["SnowDepth_cm"] = clim["Average_snow_deepth_10years"].fillna(clim["SnowAvgDepth_cm_roll3"])
clim["SnowDepth_log1p"] = np.log1p(clim["SnowDepth_cm"].clip(lower=0))

clim.head()


,Year,Week,station_id,TempMax_C,TempMin_C,Rain_mm,Average_snow_deepth_10years,SnowAvgDepth_cm_roll3,TempMean_C,DiurnalRange_C,Freeze_flag,Cold_degree,Rain_log1p,Rain_heavy,Cold_x_Rain,SnowDepth_cm,SnowDepth_log1p
0,2014,1,71032,4.714286,-2.000000,40.6,132.08,132.08,1.357143,6.714286,1,0.000000,3.728100,1,0.000000,132.08,4.89095
7,2014,2,71032,2.657143,-1.685714,8.0,132.08,132.08,0.485714,4.342857,1,0.000000,2.197225,0,0.000000,132.08,4.89095
14,2014,3,71032,-1.028571,-4.114286,4.2,132.08,132.08,-2.571429,3.085714,1,2.571429,1.648659,0,4.239408,132.08,4.89095
21,2014,4,71032,-1.314286,-4.357143,0.4,132.08,132.08,-2.835714,3.042857,1,2.835714,0.336472,0,0.954139,132.08,4.89095
28,2014,5,71032,-1.685714,-5.185714,3.0,132.08,132.08,-3.435714,3.500000,1,3.435714,1.386294,0,4.762911,132.08,4.89095


In [207]:
SEASON_WEEKS = clim["Week"].max()
clim["week_sin"] = np.sin(2*np.pi*clim["Week"]/SEASON_WEEKS)
clim["week_cos"] = np.cos(2*np.pi*clim["Week"]/SEASON_WEEKS)

clim["is_early"] = (clim["Week"] <= 5).astype(int)
clim["is_peak"]  = clim["Week"].between(6,10).astype(int)
clim["is_late"]  = (clim["Week"] >= 11).astype(int)


In [208]:
panel = vis_long.copy()   # columns: Ski Resort, Year, Week, visitors

panel = panel.sort_values(["Ski Resort","Year","Week"])
for col in ["visitors","SnowDepth_cm","TempMean_C","Rain_mm"]:
    if col in panel.columns:
        panel[f"{col}_lag1"] = panel.groupby("Ski Resort")[col].shift(1)
        panel[f"{col}_roll3"] = panel.groupby("Ski Resort")[col].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)


In [211]:
clim.columns

Index(['Year', 'Week', 'station_id', 'TempMax_C', 'TempMin_C', 'Rain_mm',
       'Average_snow_deepth_10years', 'SnowAvgDepth_cm_roll3', 'TempMean_C',
       'DiurnalRange_C', 'Freeze_flag', 'Cold_degree', 'Rain_log1p',
       'Rain_heavy', 'Cold_x_Rain', 'SnowDepth_cm', 'SnowDepth_log1p',
       'week_sin', 'week_cos', 'is_early', 'is_peak', 'is_late'],
      dtype='object')

In [212]:
cleaned_dist.columns

Index(['Ski Resort', 'State', 'Distance_MEL_km', 'Distance_SYD_km',
       'Distance_CAN_km', 'Time_from_MEL', 'Time_from_SYD', 'Time_from_CAN',
       'has_public_transport'],
      dtype='object')

In [213]:
panel.columns

Index(['Year', 'Week', 'Ski Resort', 'visitors', 'visitors_lag1',
       'visitors_roll3'],
      dtype='object')

# Exploratory Data Analysis

In [214]:
# ===== Cell 1: copy inputs (so originals stay intact) =====
vis = panel.copy()            # ['Year','Week','Ski Resort','visitors', ...]
cl  = clim.copy()             # weekly climate by station_id
dist = cleaned_dist.copy()    # distances & time strings


In [216]:
# ===== Cell 2: map resort -> station_id so we can merge climate =====
# Fill this mapping with your actual station IDs
RESORT_TO_STATIONID = {
    "Falls Creek": 71032,
    "Mount Buller": 71075,
    "Mount Hotham": 72161,
    "Mount Baw Baw": 83024,   # <-- replace with your true ID if different
    "Mount Stirling": 83084,  # <-- replace
    "Thredbo": 83024,         # example; replace with your true ID
    "Perisher": 83085,
    "Charlotte Pass": 83084,  # example; replace
    "Selwyn": 85291           # example; replace
}

vis["station_id"] = vis["Ski Resort"].map(RESORT_TO_STATIONID)


In [217]:
# ===== Cell 3: merge panel + climate (weekly) =====
df = vis.merge(
    cl, on=["Year","Week","station_id"], how="left"
)

# merge distances on resort
df = df.merge(dist, on="Ski Resort", how="left")

# sort for any rolling/lag ops later
df = df.sort_values(["Ski Resort","Year","Week"]).reset_index(drop=True)

display(df.head())


,Year,Week,Ski Resort,visitors,visitors_lag1,visitors_roll3,station_id,TempMax_C,TempMin_C,Rain_mm,...,is_peak,is_late,State,Distance_MEL_km,Distance_SYD_km,Distance_CAN_km,Time_from_MEL,Time_from_SYD,Time_from_CAN,has_public_transport
0,2014,1,Charlotte Pass,408.0,NaN,408.0,83084.0,4.600000,-0.300000,23.8,...,0.0,0.0,NSW,616.0,495.0,220.0,7.7,5.8,2.8,0.0
1,2014,2,Charlotte Pass,151.0,408.0,279.5,83084.0,4.414286,-0.285714,7.6,...,0.0,0.0,NSW,616.0,495.0,220.0,7.7,5.8,2.8,0.0
2,2014,3,Charlotte Pass,230.0,151.0,263.0,83084.0,0.100000,-2.828571,69.6,...,0.0,0.0,NSW,616.0,495.0,220.0,7.7,5.8,2.8,0.0
3,2014,4,Charlotte Pass,1134.0,230.0,505.0,83084.0,1.385714,-2.685714,24.2,...,0.0,0.0,NSW,616.0,495.0,220.0,7.7,5.8,2.8,0.0
4,2014,5,Charlotte Pass,3403.0,1134.0,1589.0,83084.0,-0.728571,-3.757143,13.6,...,0.0,0.0,NSW,616.0,495.0,220.0,7.7,5.8,2.8,0.0


In [218]:
# ===== Cell 4: parse time strings -> decimal hours & derive speeds =====
def to_hours(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    h = int(re.search(r'(\d+)\s*h', s).group(1)) if re.search(r'(\d+)\s*h', s) else 0
    m = int(re.search(r'(\d+)\s*m', s).group(1)) if re.search(r'(\d+)\s*m', s) else 0
    return h + m/60.0

for city in ["MEL","SYD","CAN"]:
    tcol_txt = f"Time_from_{city}"
    if tcol_txt in df.columns:
        df[f"Time_{city}_hrs"] = df[tcol_txt].apply(to_hours)
        dcol = f"Distance_{city}_km"
        if dcol in df.columns:
            df[f"Speed_{city}_kmh"] = df[dcol] / df[f"Time_{city}_hrs"].replace(0, np.nan)

# best access among cities
time_cols = [c for c in ["Time_MEL_hrs","Time_SYD_hrs","Time_CAN_hrs"] if c in df.columns]
dist_cols = [c for c in ["Distance_MEL_km","Distance_SYD_km","Distance_CAN_km"] if c in df.columns]

if time_cols:
    df["Time_best_hrs"] = df[time_cols].min(axis=1)
    df["Nearest_city"]  = df[time_cols].idxmin(axis=1).str.split("_").str[1]  # MEL/SYD/CAN
if dist_cols:
    df["Distance_best_km"] = df[dist_cols].min(axis=1)

df["has_public_transport"] = df["has_public_transport"].fillna(0).astype(int)

display(df.filter(regex="^Time_.*hrs$|^Speed_.*kmh$|^Distance_.*km$|Time_best_hrs|Distance_best_km|Nearest_city").head())


/var/folders/jl/8w6ytgwj7xb3r85_f95q2h8c0000gn/T/ipykernel_49813/3545071438.py:23: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df["Nearest_city"]  = df[time_cols].idxmin(axis=1).str.split("_").str[1]  # MEL/SYD/CAN


,Distance_MEL_km,Distance_SYD_km,Distance_CAN_km,Time_MEL_hrs,Speed_MEL_kmh,Time_SYD_hrs,Speed_SYD_kmh,Time_CAN_hrs,Speed_CAN_kmh,Time_best_hrs,Nearest_city,Distance_best_km
0,616.0,495.0,220.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,MEL,220.0
1,616.0,495.0,220.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,MEL,220.0
2,616.0,495.0,220.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,MEL,220.0
3,616.0,495.0,220.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,MEL,220.0
4,616.0,495.0,220.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,MEL,220.0


In [225]:
# drop rows where Year/Week/Visitors are missing
vis_long = visitation.dropna(subset=["Year","Week","Visitors"])

# ensure numeric types
vis_long["Year"] = vis_long["Year"].astype(int)
vis_long["Week"] = vis_long["Week"].astype(int)
vis_long["Visitors"] = vis_long["Visitors"].astype(int)

# optional: reset index after cleaning
vis_long = vis_long.reset_index(drop=True)

KeyError: ['Visitors']

In [223]:
annual_vis = (
    visitation
    .groupby("Year")["Visitors"]
    .sum()
    .reset_index()
)
print(annual_vis.head())

KeyError: 'Column not found: Visitors'

# Modelling

In [ ]:
def build_ml_matrix(panel):
    """Select features/target; drop leakage; get X, y, groups."""
    target = panel["visitors"].astype(float)

    # Candidate features (adjust)
    cols = [
        "TempMin_C","TempMax_C","Rain_mm","SnowMakeDepth",
        "TotalSnow_cm","AvgBase_cm","MaxBase_cm","MaxStorm_cm",
        "lift","blue_slope","red_slope","black_slope","cross_country",
        "Distance_MEL_km","Distance_SYD_km","Distance_CAN_km",
        "Time_MEL_min","Time_SYD_min","Time_CAN_min","has_public_transport",
        "visitors_lag1","TempMin_C_lag1","TempMax_C_lag1","Rain_mm_lag1",
        "visitors_roll3","TempMin_C_roll3","Rain_mm_roll3"
    ]
    cols = [c for c in cols if c in panel.columns]
    X = panel[cols].copy()
    # Optional: impute/scale here
    return X, target, cols

In [ ]:
def split_time(panel, X, y):
    """Train/test split by last season or similar (adjust to your years)."""
    # Example split: use max Year as test
    test_year = panel["Year"].max()
    train_idx = panel["Year"] < test_year
    test_idx  = panel["Year"] == test_year
    return train_idx, test_idx, test_year

In [ ]:
def train_baselines(X_tr, y_tr):
    """Fit a simple baseline model (e.g., linear regression)."""
    # model = LinearRegression()
    # model.fit(X_tr, y_tr)
    # return {"linreg": model}
    return {}

In [ ]:
def train_xgb(X_tr, y_tr):
    """Fit XGBoost if available."""
    # if xgb is None:
    #     return {}
    # dtrain = xgb.DMatrix(X_tr, label=y_tr)
    # params = {"max_depth":4, "eta":0.1, "subsample":0.9, "colsample_bytree":0.9,
    #           "objective":"reg:squarederror", "seed":SEED}
    # bst = xgb.train(params, dtrain, num_boost_round=400)
    # return {"xgb": bst}
    return {}

In [ ]:
def evaluate_models(models, X_tr, y_tr, X_te, y_te):
    """Compute RMSE/MAE/R2; add feature importance where applicable."""
    results = []
    # Example for linear model:
    # if "linreg" in models:
    #     m = models["linreg"]
    #     yhat_tr = m.predict(X_tr)
    #     yhat_te = m.predict(X_te)
    #     results.append(("linreg","train",
    #         float(np.sqrt(mean_squared_error(y_tr, yhat_tr))),
    #         float(mean_absolute_error(y_tr, yhat_tr)),
    #         float(r2_score(y_tr, yhat_tr))))
    #     results.append(("linreg","test",
    #         float(np.sqrt(mean_squared_error(y_te, yhat_te))),
    #         float(mean_absolute_error(y_te, yhat_te)),
    #         float(r2_score(y_te, yhat_te))))
    # TODO: add xgb evaluation similarly
    return pd.DataFrame(results, columns=["model","split","RMSE","MAE","R2"])


# Positioning and Clustering

In [ ]:
def build_resort_profiles(features_df, distances_df, snowfall_season_df):
    """One row per resort with static traits → for clustering/positioning."""
    df = features_df.merge(distances_df, on="Ski Resort", how="left")
    if snowfall_season_df is not None:
        # average across years
        snow_prof = (snowfall_season_df.groupby("Ski Resort")
                        [["TotalSnow_cm","AvgBase_cm","MaxBase_cm","MaxStorm_cm"]].mean().reset_index())
        df = df.merge(snow_prof, on="Ski Resort", how="left")
    # Optional: scale/standardize before KMeans
    return df

In [ ]:
def cluster_resorts(profile_df, k=3):
    """K-means cluster to find peer groups (adjust K)."""
    # feats = profile_df.select_dtypes(include=[np.number]).fillna(0).values
    # scaler = StandardScaler()
    # Z = scaler.fit_transform(feats)
    # km = KMeans(n_clusters=k, random_state=SEED, n_init="auto").fit(Z)
    # profile_df["cluster"] = km.labels_
    return profile_df

# Scenario Analysis

In [ ]:
def run_scenarios(panel, model, cols):
    """What-if: +1°C warmer, +20% rainfall, improved transport, etc."""
    scenarios = []

    # Example: +1°C min/max
    # df_warm = panel.copy()
    # for c in ["TempMin_C","TempMax_C"]:
    #     if c in df_warm.columns: df_warm[c] = df_warm[c] + 1.0
    # y_base = model.predict(panel[cols])
    # y_warm = model.predict(df_warm[cols])
    # delta = (y_warm - y_base).mean()
    # scenarios.append({"scenario":"+1C temps", "avg_delta_visitors": float(delta)})

    return pd.DataFrame(scenarios)

In [ ]:
def main(visitation, climate, features, vic_snowfall, nsw_snowfall, distances):
    # --- Clean ---
    vis_long  = clean_visitation(visitation)
    clim_wk   = clean_climate(climate)
    feats     = clean_features(features)
    dists     = clean_distances(distances)
    snow_vic  = clean_snowfall_vic(vic_snowfall) if vic_snowfall is not None else None
    snow_nsw  = clean_snowfall_nsw(nsw_snowfall) if nsw_snowfall is not None else None

    # --- Panel ---
    panel = make_weekly_panel(vis_long, clim_wk, snow_vic, snow_nsw, feats, dists)

    # --- EDA ---
    eda = run_eda(panel)

    # --- ML Matrix ---
    X, y, cols = build_ml_matrix(panel)
    train_idx, test_idx, test_year = split_time(panel, X, y)
    X_tr, y_tr = X[train_idx], y[train_idx]
    X_te, y_te = X[test_idx], y[test_idx]

    # --- Models ---
    models = {}
    models.update(train_baselines(X_tr, y_tr))
    models.update(train_xgb(X_tr, y_tr))
    eval_df = evaluate_models(models, X_tr, y_tr, X_te, y_te)

    # Pick a champion model (your logic here)
    # champion = models.get("xgb") or models.get("linreg")

    # --- Positioning / Clusters ---
    profile_df = build_resort_profiles(feats, dists, snow_nsw)
    profile_df = cluster_resorts(profile_df, k=3)

    # --- Scenarios ---
    # scenarios_df = run_scenarios(panel, champion, cols) if champion else pd.DataFrame()

    outputs = {
        "panel": panel,
        "eda": eda,
        "eval": eval_df,
        "profiles": profile_df,
        # "scenarios": scenarios_df,
        "test_year": test_year
    }
    return outputs

# ===============================
# 9) Usage (you already have DataFrames in memory)
# ===============================
# results = main(visitation, climate, features, vic_snowfall, nsw_snowfall, distances)
# results["eval"].head()
# results["eda"]["resort_summary"].head()
# results["profiles"].head()

In [ ]:
results = main(
    visitation, 
    climate, 
    features, 
    vic_snowfall, 
    nsw_snowfall, 
    distances
)


NameError: name 'features' is not defined